In [0]:
# HTTP
from requests import get

# OS and IO
from io import BytesIO
import os
import shutil
import glob
from pathlib import Path

# Unzipping
from zipfile import ZipFile

# Actual thing
import scipy
import matplotlib.pyplot as plt

# For sorting files
from natsort import natsorted

# Numpy
import numpy as np

request = get('http://www.cl.cam.ac.uk/Research/DTG/attarchive/pub/data/att_faces.zip')
zip_file = ZipFile(BytesIO(request.content))
files = zip_file.namelist()
files = natsorted(files)
zip_file.extractall()

dir_path = os.getcwd()

#pathlist = Path(dir_path).glob('**/*.pgm')
#for path in pathlist:
 #   path_in_str = str(path)
  #  print(path_in_str)

# BUGGYYY
#images = [path for path in pathlist]
#images = sorted(images, key=lambda x: str(x))

#print(images)

# Reading the first file in the zip file:
#zip_file.read(files[0])
# sorted_files = sorted(files)

# Because the matrix is at first empty, we append the first instance, afterwards we push the images vertically, I think we can do it in a cleaner way but i'm a bit lazy now
# The array is transposed because I checked its dimensions and they were 112X92
flag = 0
y = np.arange(40).reshape(40, 1)
ytest = y[1::2, ::]
ytrainning = y[0::2, ::]
D = np.array([[]])
for f in files:
    if f.endswith(".pgm"):
        arr = scipy.misc.imread(f)
        arr = arr.T
        if flag == 0:
            D = np.append(D,arr.flatten())
            flag = 1
        else:
            D = np.vstack((D, arr.flatten()))
Dtest = np.copy(D[1::2, ::])
Dtrainning = D[0::2, ::]



In [44]:
means = np.array(np.empty([0, 10304]))
j = 0
for i in range(0, 400, 10):
    user = D[i:i+10:, ::]
    means = np.vstack((means, np.mean(user, axis=0)))

[47.8 47.2 52.1 ... 49.5 49.2 49. ]
